In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http:

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()



--2022-08-28 21:19:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-08-28 21:19:14 (11.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



***Import data from SQL table and Create a Dataframe***

In [3]:
# Store environmental variable

from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS

mode = "append"
jdbc_url="jdbc:postgresql://challenge.cib8i0dtf6rx.us-east-1.rds.amazonaws.com:5432/Challenge_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [7]:
vine_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)

In [8]:
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

***Filter Data from Vine Table Dataframe***

In [23]:
#1. Filter data and create a new df to retrieve all the rows where total_votes vount >= 20.

total_votes_df = vine_df[vine_df['total_votes'] >= 20]

In [24]:
total_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [25]:
#2. filter new df and create new df to retreieve all rows where the number of helpful_votes/total_votes >= 50%.

helpful_votes = total_votes_df[total_votes_df["helpful_votes"]/total_votes_df["total_votes"]>=0.5]

In [26]:
helpful_votes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [32]:
#3. Filter the df and create a new df that retrieves all the rows ehere a review was written as part of vine program(paid)
#vine == "Y"

paid_vine_df = helpful_votes[helpful_votes["vine"] == "Y"]

In [33]:
paid_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [34]:
#4. Repeat step 3 but retrieve where review was not part of vine program
#vine == N

non_paid_vine_df = helpful_votes[helpful_votes["vine"] == "N"]

In [35]:
non_paid_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

***Analysis of Filtered Table***

In [87]:
#create function for percentage
def get_percentage(decimal):
    convert = "{:.0%}".format(decimal)

    return convert;

In [91]:
#5. Paid Vine analysis:

#total number of review:
count_paid_review = paid_vine_df.count()

#count number of 5 star reviews
five_star_count = paid_vine_df[paid_vine_df["star_rating"]== 5].count()

# percentage of 5-star reviews
percent_five_star_count = five_star_count/count_paid_review


print(f"Total count of paid Vine reviews: {count_paid_review} \n" 
      f"Total Count five-star reviews from paid Vine users: {five_star_count} \n"
      f"Percentage of five-star reviews from paid Vine users: {get_percentage(percent_five_star_count)}")

Total count of paid Vine reviews: 94 
Total Count five-star reviews from paid Vine users: 48 
Percentage of five-star reviews from paid Vine users: 51%


In [92]:
#5. non Paid analysis:

#total number of review:
count_non_paid_review = non_paid_vine_df.count()

#count number of 5 star reviews
five_star_non_count = non_paid_vine_df[non_paid_vine_df["star_rating"]== 5].count()

# percentage of 5-star reviews
percent_five_star_non_count = five_star_non_count/count_non_paid_review

print(f"Total count of non-paid Vine reviews: {count_non_paid_review} \n" 
      f"Total count five-star reviews from non-paid Vine users: {five_star_non_count} \n"
      f"Percentage of five-star reviews from non-paid Vine users: {get_percentage(percent_five_star_non_count)}")

Total count of non-paid Vine reviews: 40471 
Total count five-star reviews from non-paid Vine users: 15663 
Percentage of five-star reviews from non-paid Vine users: 39%
